In [16]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [20]:
!pip install --upgrade pip
!pip install pyathena

    100% |████████████████████████████████| 1.3MB 9.5MB/s ta 0:00:01
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


Files are retrieved from [here](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml) and named like this: https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-01.csv

Those files are stored in S3. https://registry.opendata.aws/nyc-tlc-trip-records-pds/

In [13]:
GREEN_RAW_LOCAL  = '~/data/nyc_green/'
YELLOW_RAW_LOCAL = '~/data/nyc_yellow/'
STAGING_BUCKET = 's3://mkamp-aws-tmp/nyc_tmp'

In [60]:
%%time
!echo {YELLOW_RAW_LOCAL}
!mkdir -p {GREEN_RAW_LOCAL}
!mkdir -p {YELLOW_RAW_LOCAL}
!ls -l ~/data/

print('ref:')
!aws s3 sync 's3://nyc-tlc/misc/' {STAGING_BUCKET}/ref --exclude="*" --include='taxi _zone_lookup.csv' 

print('green:')
!aws s3 sync 's3://nyc-tlc/trip data/' {STAGING_BUCKET}/green --exclude="*" --include='green_tripdata_2016*.csv' --quiet
!aws s3 sync 's3://nyc-tlc/trip data/' {STAGING_BUCKET}/green --exclude="*" --include='green_tripdata_2017*.csv' --quiet
!aws s3 sync 's3://nyc-tlc/trip data/' {STAGING_BUCKET}/green --exclude="*" --include='green_tripdata_2018*.csv' 

print('yellow:')
!aws s3 sync 's3://nyc-tlc/trip data/' {STAGING_BUCKET}/yellow --exclude="*" --include='yellow_tripdata_2016*.csv' --quiet
!aws s3 sync 's3://nyc-tlc/trip data/' {STAGING_BUCKET}/yellow --exclude="*" --include='yellow_tripdata_2017*.csv' --quiet
!aws s3 sync 's3://nyc-tlc/trip data/' {STAGING_BUCKET}/yellow --exclude="*" --include='yellow_tripdata_2018*.csv' 

!aws s3 ls {STAGING_BUCKET}/ref/

/home/ec2-user/data/nyc_yellow/
total 8
drwxrwxr-x 2 ec2-user ec2-user 4096 Oct 21 14:05 nyc_green
drwxrwxr-x 2 ec2-user ec2-user 4096 Oct 21 14:05 nyc_yellow
ref:
green:
yellow:
2018-10-21 15:35:32      12322 taxi _zone_lookup.csv
CPU times: user 142 ms, sys: 84.2 ms, total: 226 ms
Wall time: 10.1 s


In [21]:
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor

In [64]:
CREATE_REF_SQL = """
CREATE EXTERNAL TABLE `taxi_zones`(
  `locationid` bigint, 
  `borough` string, 
  `zone` string, 
  `service_zone` string)
ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://mkamp-aws-tmp/nyc_tmp/ref'
TBLPROPERTIES (
  'classification'='csv', 
  'columnsOrdered'='true', 
  'compressionType'='none', 
  'typeOfData'='file')
"""

CREATE_YELLOW_SQL = """
CREATE EXTERNAL TABLE `raw_yellow`(
  `vendorid` bigint, 
  `lpep_pickup_datetime` string, 
  `lpep_dropoff_datetime` string, 
  `passenger_count` bigint, 
  `trip_distance` double, 
  `ratecodeid` bigint, 
  `store_and_forward_flag` string, 
  `pu_locationid` bigint, 
  `do_locationid` bigint, 
  `payment_type` bigint, 
  `fare_amount` double, 
  `extra` double, 
  `mta_tax` double, 
  `tip_amount` double, 
  `tolls_amount` double, 
  `improvement_surcharge` double, 
  `total_amount` double)
ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://mkamp-aws-tmp/nyc_tmp/yellow'
TBLPROPERTIES (
  'classification'='csv', 
  'columnsOrdered'='true', 
  'compressionType'='none', 
  'skip.header.line.count'='1',
  'typeOfData'='file')
"""

CREATE_GREEN_SQL = """
CREATE EXTERNAL TABLE `raw_green`(
  `vendorid` bigint, 
  `lpep_pickup_datetime` string, 
  `lpep_dropoff_datetime` string, 
  `store_and_forward_flag` string, 
  `ratecodeid` bigint, 
  `pu_locationid` bigint, 
  `do_locationid` bigint, 
  `passenger_count` bigint, 
  `trip_distance` double, 
  `fare_amount` double, 
  `extra` double, 
  `mta_tax` double, 
  `tip_amount` double, 
  `tolls_amount` double, 
  `ehail_fee` string, 
  `improvement_surcharge` double, 
  `total_amount` double, 
  `payment_type` bigint, 
  `trip_type` bigint)
ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://mkamp-aws-tmp/nyc_tmp/green'
TBLPROPERTIES (
  'classification'='csv', 
  'columnsOrdered'='true', 
  'compressionType'='none', 
  'skip.header.line.count'='1',
  'typeOfData'='file')
"""

In [71]:
%%time
connection = connect(s3_staging_dir='s3://mkamp-fastai/staging/',
                 region_name='eu-west-1')
print(connection)
cursor = connection.cursor()
print(cursor)
print('ref result:', cursor.execute(CREATE_REF_SQL))
print('yellow result:', cursor.execute(CREATE_YELLOW_SQL))
print('green result:', cursor.execute(CREATE_GREEN_SQL))

OperationalError: FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.DDLTask. AlreadyExistsException(message:Table taxi_zones already exist)

In [72]:
PREPARE_RAW_TRIPS_SQL = """
CREATE OR REPLACE VIEW "raw_trips" AS
WITH 
  yellow_trips AS (
    SELECT
      CAST(parse_datetime(lpep_pickup_datetime, 'YYYY-MM-dd HH:mm:ss') AS TIMESTAMP) pu_dt,
      CAST(parse_datetime(lpep_dropoff_datetime, 'YYYY-MM-dd HH:mm:ss') AS TIMESTAMP) do_dt,
      'yellow' source,
      pu_locationid,
      do_locationid,
      trip_distance,
      fare_amount
    FROM 
      raw_yellow),
  green_trips AS (
    SELECT
      CAST(parse_datetime(lpep_pickup_datetime, 'YYYY-MM-dd HH:mm:ss') AS TIMESTAMP) pu_dt,
      CAST(parse_datetime(lpep_dropoff_datetime, 'YYYY-MM-dd HH:mm:ss') AS TIMESTAMP) do_dt,
      'green' source,
      pu_locationid,
      do_locationid,
      trip_distance,
      fare_amount
    FROM 
      raw_green)
SELECT * FROM green_trips
WHERE
  pu_dt >= timestamp '2016-01-01 00:00 UTC' AND
  pu_dt <  timestamp '2018-07-01 00:00 UTC' 
UNION
SELECT * FROM yellow_trips
WHERE
  year(pu_dt) in (2016, 2017, 2018)
"""
print('prepare raw trips sql:', cursor.execute(PREPARE_RAW_TRIPS_SQL))

OperationalError: Query timeout

In [66]:
CREATE_PREPARED_TRIPS_SQL = """
CREATE OR REPLACE VIEW prepared_trips AS
SELECT
  rt.pu_dt,
  rt.do_dt,
  coalesce(tz_pu.borough, 'Data Missing') pu_borough,
  coalesce(tz_pu.zone, 'Data Missing') pu_zone,
  coalesce(tz_do.borough, 'Data Missing') do_borough,
  coalesce(tz_do.zone, 'Data Missing') do_zone,
  rt.source,
  rt.trip_distance,
  rt.fare_amount,
  CAST((to_unixtime(rt.do_dt) - to_unixtime(rt.pu_dt)) AS INTEGER) trip_duration_s
FROM 
  raw_trips rt
  LEFT OUTER JOIN taxi_zones tz_pu ON rt.pu_locationid = tz_pu.locationid
  LEFT OUTER JOIN taxi_zones tz_do ON rt.do_locationid = tz_do.locationid
"""
print('CREATE_PREPARED_TRIPS_SQL:', cursor.execute(CREATE_PREPARED_TRIPS_SQL))

CREATE_PREPARED_TRIPS_SQL: <pyathena.cursor.Cursor object at 0x7fa69dbcff98>


In [67]:
CREATE_PREPARED_TRIPS_AGGREGATED_SQL = """
CREATE OR REPLACE VIEW prepared_trips_aggregated AS 
SELECT
  date_trunc('hour', pu_dt) pu_dt, 
  year(pu_dt)  pu_year,
  month(pu_dt) pu_month,
  day(pu_dt)   pu_day,
  hour(pu_dt)  pu_hour,
  pu_borough,
  pu_zone,
  source,
  COUNT(*) freq
FROM 
  prepared_trips
WHERE 
  pu_borough != 'Data Missing' AND
  pu_borough != 'Data Missing'
GROUP BY 1, 2, 3, 4, 5, 6, 7, 8
ORDER BY 1 desc
"""
print('CREATE_PREPARED_TRIPS_AGGREGATED_SQL:', cursor.execute(CREATE_PREPARED_TRIPS_AGGREGATED_SQL))

CREATE_PREPARED_TRIPS_AGGREGATED_SQL: <pyathena.cursor.Cursor object at 0x7fa69dbcff98>


In [79]:
%%time
SQL = """
CREATE TABLE prepared_trips_aggregated_materialized 
WITH (
  format='Parquet',
  parquet_compression = 'SNAPPY',
  external_location='s3://mkamp-aws-tmp/nyc_tmp/prepared/',
  partitioned_by = ARRAY['pu_year', 'pu_month']) 
AS
SELECT 
   pu_dt,
   pu_hour,
   pu_borough,
   pu_zone,
   source,
   freq,
   pu_year,
   pu_month
FROM 
  prepared_trips_aggregated
ORDER BY 1 desc
"""
print('MATERIALIZE AGGREGATE', cursor.execute(SQL))
print(cursor.data_scanned_in_bytes/1024**3) # GB

MATERIALIZE AGGREGATE <pyathena.cursor.Cursor object at 0x7fa69e1c5320>
CPU times: user 279 ms, sys: 0 ns, total: 279 ms
Wall time: 1min 39s


In [53]:
%%time

cursor = connect(s3_staging_dir='s3://mkamp-fastai/staging/',
                 region_name='eu-west-1',
                 cursor_class=PandasCursor).cursor()

df_raw = cursor.execute("SELECT * FROM raw_trips LIMIT 10").as_pandas()
print(df_raw)

                pu_dt               do_dt  source  pu_locationid  \
0 2017-05-16 16:57:32 2017-05-16 17:02:05  yellow            141   
1 2017-05-16 16:57:32 2017-05-16 17:08:15  yellow            113   
2 2017-05-16 16:57:32 2017-05-16 17:02:17  yellow            236   
3 2017-05-16 16:57:33 2017-05-16 17:34:54  yellow            132   
4 2017-05-16 16:57:33 2017-05-16 17:00:12  yellow            239   
5 2017-05-16 16:57:33 2017-05-16 17:01:17  yellow            262   
6 2017-05-16 16:57:33 2017-05-17 16:54:58  yellow            142   
7 2017-05-16 16:57:33 2017-05-16 17:25:03  yellow            229   
8 2017-05-16 16:57:33 2017-05-16 17:24:27  yellow              4   
9 2017-05-16 16:57:33 2017-05-16 17:06:52  yellow            170   

   do_locationid  trip_distance  fare_amount  
0            263           1.22          5.5  
1            114           0.53          7.5  
2            237           0.60          5.0  
3             95           8.30         32.0  
4            238

SELECT pu_year, pu_month, sum(freq)/1000000.0 FROM default.prepared_trips_aggregated_materialized 
GROUP BY 1, 2 
ORDER BY 1, 2
